# Фреймворк PyTorch для разработки искусственных нейронных сетей

## Урок 9. Трансформер

### Практическое задание

1. Возьмите готовую модель из https://huggingface.co/models для классификации сентимента текста.
2. Сделайте предсказания на всем df_val. Посчитайте метрику качества.
3. Дообучите эту модель на df_train. Посчитайте метрику качества на df_val.

Данные на google drive: https://drive.google.com/file/d/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901

### Решение

#### Импорт библиотек

In [76]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from transformers import BertTokenizer
from transformers import BertForSequenceClassification

from tqdm import tqdm

#### Настройки проекта

In [2]:
# Случайное зерно.
GLOBAL__RANDOM_STATE = 0

# Выбор устройства.
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Путь к данным.
PATH__DATA_TRAIN = r'train.csv'
PATH__DATA_VAL = r'val.csv'

# Выбор устройства.
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#### 1. Выбор готовой модели

Для выбора модели использовались следующие критерии:

    Язык: русский
    Библиотека: PyTorch
    Задача: классификация текста

https://huggingface.co/models?language=ru&library=pytorch&pipeline_tag=text-classification&sort=downloads

В результате была выбрана модель `SkolkovoInstitute/russian_toxicity_classifier`, построенная на основе классической модели `Bert`.

https://huggingface.co/SkolkovoInstitute/russian_toxicity_classifier

In [3]:
%%time

# Загрузка токенайзера модели.
tokenizer = BertTokenizer.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')

CPU times: total: 172 ms
Wall time: 2.23 s


In [4]:
%%time

# Загрузка модели и весов.
model = BertForSequenceClassification.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')

CPU times: total: 1.97 s
Wall time: 2.25 s


In [5]:
# Проверка работы модели на фразе без оскорбления.
batch = tokenizer.encode('ты супер', return_tensors='pt')
model(batch)

SequenceClassifierOutput(loss=None, logits=tensor([[ 4.0289, -3.7206]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [6]:
# Проверка работы модели на фразе с оскорблением.
batch = tokenizer.encode('ты дурак', return_tensors='pt')
model(batch)

SequenceClassifierOutput(loss=None, logits=tensor([[-2.1764,  1.0085]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

Модель принимает на вход ембеддинг собственного токенайзера, а возвращает тензор с двумя значениями.

У модели отсутствует сопроводительная документация, но тестирование показало, что значения тензора меняют знак в зависимости от эмоциональной окраски.

Выполним предсказание на валидационной выборке чтобы оценить качество.

#### Подготовка данных

In [7]:
# Загрузка обучающей выборки.
df_train = pd.read_csv(PATH__DATA_TRAIN)

In [8]:
# Проверка: вывод первых пяти строк обучающей выборки.
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [9]:
# Класс датасета.
class TwitterDataset(Dataset):
    
    def __init__(self, txts, labels):
        self._labels = labels
        
        self.tokenizer = BertTokenizer.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')
        self._txts = [
            self.tokenizer.encode(
                text,
                padding='max_length',
                max_length=10,
                truncation=True,
                return_tensors="pt"
            )[0] for text in txts
        ]
        
    def __len__(self):
        return len(self._txts)
    
    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [10]:
%%time

# Подготовка обучающего датасета.
ds_train = TwitterDataset(df_train['text'], df_train['class'])

CPU times: total: 50.3 s
Wall time: 52.5 s


In [11]:
# Подготовка обучающего загрузчика.
dl_train = DataLoader(ds_train, batch_size=64, shuffle=True)

In [12]:
# Проверка обучающего загрузчика.
for x, y in dl_train:
    print(f'Размерность тензора объекта:\t{x[0].shape}')
    print(f'Тензор:\n\t{x[0]}')
    print(f'Размерность тензора классов:\t{y.shape}')
    print(f'Тензор классов:\n\t{y}')
    break

Размерность тензора объекта:	torch.Size([10])
Тензор:
	tensor([  101,   108,   168,   186, 12392,   171, 12642, 15113,  4135,   102])
Размерность тензора классов:	torch.Size([64])
Тензор классов:
	tensor([0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1,
        0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0,
        1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1])


In [13]:
# Загрузка валидационной выборки.
df_val = pd.read_csv(PATH__DATA_VAL)

In [14]:
# Проверка: вывод первых пяти строк валидационной выборки.
df_val.head()

,id,text,class
0,181467,RT @TukvaSociopat: Максимальный репост! ))) #є...,1
1,181468,чтоб у меня з.п. ежегодно индексировали на инд...,0
2,181469,@chilyandlime нехуя мне не хорошо !!! :((((,0
3,181470,"@inafish нее , когда ногами ахахах когда?ахаха...",0
4,181471,"Хочу сделать как лучше, а получаю как всегда. :(",0


In [15]:
%%time

# Подготовка валидационного датасета.
ds_val = TwitterDataset(df_val['text'], df_val['class'])

CPU times: total: 7.08 s
Wall time: 8.31 s


In [16]:
# Подготовка валидационного загрузчика.
dl_val = DataLoader(ds_val, batch_size=64, shuffle=True)

In [17]:
# Проверка валидационного загрузчика.
for x, y in dl_val:
    print(f'Размерность тензора объекта:\t{x[0].shape}')
    print(f'Тензор:\n\t{x[0]}')
    print(f'Размерность тензора классов:\t{y.shape}')
    print(f'Тензор классов:\n\t{y}')
    break

Размерность тензора объекта:	torch.Size([10])
Тензор:
	tensor([  101,   168, 10637, 11745, 10855,   267,   230, 16781, 21408,   102])
Размерность тензора классов:	torch.Size([64])
Тензор классов:
	tensor([0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
        0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1])


#### Постороение модели

In [18]:
# Класс модели.
class BertClassifier(nn.Module):
    def __init__(self, dropout=0.5):
        super().__init__()
        self.bert = BertForSequenceClassification.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')
        self.sigm = nn.Sigmoid()

    def forward(self, x):
        x = self.bert(x).logits
        x = self.sigm(x).argmax(dim=1)
        return x

#### 2. Предсказание на валидационной выборке

In [19]:
# Иницализация модели.
model = BertClassifier()

In [20]:
# Вывод информации о модели.
print(model)
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))

BertClassifier(
  (bert): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(119547, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=Tr

In [21]:
%%time

# Списки для истинного и спрогнозированного класса объектов.
list_y_true = []
list_y_pred = []

# Прогноз на валидационной выборке.
for x, y in tqdm(dl_val):
    y_pred = model.forward(x)
    
    list_y_true += [*y.numpy()]
    list_y_pred += [*y_pred.numpy()]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 355/355 [01:37<00:00,  3.63it/s]

CPU times: total: 12min 16s
Wall time: 1min 37s


In [22]:
# Вывод метрик качества.
print(classification_report(y_true=list_y_true, y_pred=list_y_pred))

              precision    recall  f1-score   support

           0       0.49      0.92      0.64     11234
           1       0.43      0.06      0.10     11449

    accuracy                           0.49     22683
   macro avg       0.46      0.49      0.37     22683
weighted avg       0.46      0.49      0.37     22683



Исходная модель крайне плохо справилась с одним из классов.

#### 3. Дообучение модели

In [184]:
# Класс модели.
class BertClassifier_v2(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.bert = BertForSequenceClassification.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')
        self.bert.requires_grad_(False)
        
        self.linear_1 = nn.Linear(in_features=2, out_features=20)
        self.drop_out_1 = nn.Dropout1d(p=0.1)
        self.batch_norm_1 = nn.BatchNorm1d(num_features=20)
        self.leaky_relu_1 = nn.LeakyReLU(negative_slope=0.1)
        
        self.linear_2 = nn.Linear(in_features=20, out_features=20)
        self.drop_out_2 = nn.Dropout1d(p=0.1)
        self.batch_norm_2 = nn.BatchNorm1d(num_features=20)
        self.leaky_relu_2 = nn.LeakyReLU(negative_slope=0.1)
        
        self.linear_3 = nn.Linear(in_features=20, out_features=10)
        self.leaky_relu_3 = nn.LeakyReLU(negative_slope=0.1)
        
        self.linear_4 = nn.Linear(in_features=10, out_features=5)
        self.leaky_relu_4 = nn.LeakyReLU(negative_slope=0.1)
        
        self.linear_5 = nn.Linear(in_features=5, out_features=1)
        self.leaky_relu_5 = nn.LeakyReLU(negative_slope=0.1)
        
        self.sigm = nn.Sigmoid()
    

    def forward(self, x, threshold=0.5):
        x = self.bert(x).logits
        
        x = self.linear_1(x)
        x = self.drop_out_1(x)
        x = self.batch_norm_1(x)
        x = self.leaky_relu_1(x)
        
        x = self.linear_2(x)
        x = self.drop_out_2(x)
        x = self.batch_norm_2(x)
        x = self.leaky_relu_2(x)
        
        x = self.linear_3(x)
        x = self.leaky_relu_3(x)
        
        x = self.linear_4(x)
        x = self.leaky_relu_4(x)
        
        x = self.linear_5(x)
        x = self.leaky_relu_5(x)
        
        x = self.sigm(x)
        
        return (x > threshold).int()

In [185]:
# Иницализация модели.
model_v2 = BertClassifier_v2()

In [186]:
# Вывод информации о параметрах.
print("Parameters full train:\t", sum([param.nelement() for param in model_v2.parameters()]))
print("Parameters level 1:\t", sum([param.nelement() for param in model_v2.linear_1.parameters()]))
print("Parameters level 2:\t", sum([param.nelement() for param in model_v2.linear_2.parameters()]))
print("Parameters level 3:\t", sum([param.nelement() for param in model_v2.linear_3.parameters()]))
print("Parameters level 4:\t", sum([param.nelement() for param in model_v2.linear_4.parameters()]))
print("Parameters level 5:\t", sum([param.nelement() for param in model_v2.linear_5.parameters()]))

Parameters full train:	 177855809
Parameters level 1:	 60
Parameters level 2:	 420
Parameters level 3:	 210
Parameters level 4:	 55
Parameters level 5:	 6


In [187]:
# Отключение обучения всех параметров.
for param in model_v2.parameters():
    param.requires_grad = False

In [188]:
# Обучение первого дополнительного слоя.
for param in model_v2.linear_1.parameters():
    param.requires_grad = True

In [189]:
# Обучение второго дополнительного слоя.
for param in model_v2.linear_2.parameters():
    param.requires_grad = True

In [190]:
# Обучение третьего дополнительного слоя.
for param in model_v2.linear_3.parameters():
    param.requires_grad = True

In [191]:
# Обучение четвёртого дополнительного слоя.
for param in model_v2.linear_4.parameters():
    param.requires_grad = True

In [192]:
# Обучение пятого дополнительного слоя.
for param in model_v2.linear_5.parameters():
    param.requires_grad = True

In [193]:
# Функция потерь.
criterion = nn.CrossEntropyLoss()

# Оптимизатор.
optimizer = Adam(model_v2.parameters(), lr=0.001)  # полное обучение

In [194]:
# Обучение модели на одной эпохе.
for epoch_num in range(1):
    total_acc_train = 0
    total_loss_train = 0

    model.train()
    for train_input, train_label in tqdm(dl_train):
        output = model_v2(train_input)
                
        batch_loss = criterion(output.float().reshape(1, -1), train_label.float().reshape(1, -1))
        total_loss_train += batch_loss.item()
                
        acc = (output.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

        model_v2.zero_grad()
        batch_loss.backward()
        optimizer.step()
            
    model_v2.eval()
    total_loss_val, total_acc_val = 0.0, 0.0
    
    for val_input, val_label in valid_loader:

        output = model_v2(val_input)

        batch_loss = criterion(output.float().reshape(1, -1), val_label.float().reshape(1, -1))
        total_loss_val += batch_loss.item()
                    
        acc = (output.argmax(dim=1) == val_label).sum().item()
        total_acc_val += acc
            
    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train Accuracy: {total_acc_train / len(train_dataset): .3f} \
        | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
        | Val Accuracy: {total_acc_val / len(valid_dataset): .3f}')

  0%|                                                                                                                                                                                    | 0/2836 [00:00<?, ?it/s]


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
%%time

# Списки для истинного и спрогнозированного класса объектов.
list_y_true = []
list_y_pred = []

# Прогноз на валидационной выборке.
for x, y in tqdm(dl_val):
    y_pred = model_v2.forward(x)
    
    list_y_true += [*y.numpy()]
    list_y_pred += [*y_pred.numpy().reshape(1, -1)[0]]